<a href="https://colab.research.google.com/github/Omkar2499/Dataset/blob/main/Pothole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import os
import cv2
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf
from google.colab.patches import cv2_imshow
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [39]:
class_names = ['Black Ice','Dry Asphalt','Dry Cobblestone','Dry Concrete','Dry Gravel','Dry Mud',
               'Heavy Snow','Light Snow','Muddy Wet','Packed Snow','Refrozen Ice','Sleet','Slush',
               'Wet Asphalt','Wet Cobblestone','Wet Concrete','Wet Gravel','Wet Heavy']

class_n = ['Plain','Pothole']

In [40]:
# Define Friction Coefficients
friction_coefficients = {
    "Wet Asphalt" :"0.40-0.50",
    "Black Ice": "0.005-0.02",
    "Wet Cobblestone" : "0.30-0.40",
    "Wet Concrete" : "0.40-0.50",
    "Wet Gravel" : "0.30-0.40",
    "Heavy Snow": "0.15-0.30",
    "Wet Heavy" :  "0.30-0.50",
    "Light Snow": "0.05-0.20",
    "Muddy Wet" :   "0.20-0.30",
    "Packed Snow": "0.05-0.15",
    "Refrozen Ice": "0.30-0.40",
    "Sleet": "0.50",
    "Slush": "0.35-0.40",
    "Dry Asphalt" : "0.70-0.80",
    "Dry Cobblestone" : "0.60-0.70",
    "Dry Concrete" :  "0.70-0.80",
    "Dry Mud" :  "0.50-0.60",
    "Dry Gravel" : "0.60-0.70"
}

In [41]:
model = tf.keras.models.load_model('Github_Model.keras')
model1 = tf.keras.models.load_model('Road_Surface_Classification_Video_Based.keras')

In [ ]:
input_video_path = '/content/istockphoto-1800687328-640_adpp_is.mp4'
output_video_path = '/content/istockphoto-1800687328-640_adpp_is_output.mp4'
dashboard_image_path = '/content/Dashboard.png'  # Dashboard path

# Load the dashboard image
dashboard = cv2.imread(dashboard_image_path, cv2.IMREAD_UNCHANGED)

# Check if the image has 4 channels (RGBA)
if dashboard.shape[-1] == 4:
    # Convert the image from RGBA to RGB
    dashboard = cv2.cvtColor(dashboard, cv2.COLOR_BGRA2BGR)

# Resize the dashboard to fit the video frame (adjust dimensions as needed)
dashboard_height, dashboard_width = 110, 235  # Adjust the size if needed
dashboard = cv2.resize(dashboard, (dashboard_width, dashboard_height))

cap = cv2.VideoCapture(input_video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

class_data = []
class_names_data = []
confidences_data = []
mu_values_data = []
slip_ratios_data = []
friction_data_EKF = []
friction_data = []

# Font settings for displaying text
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.27
font_color = (255, 255, 255)  # White text
thickness = 1

vehicle_speed = 20.0  # m/s (example value)
tire_angular_velocity = 10.0  # rad/s (example value)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Overlay the dashboard at the top-left corner of the frame
    frame[0:dashboard_height, 0:dashboard_width] = dashboard

    # Prediction road surface type for a frame
    resized_frame = cv2.resize(frame, (224, 224))
    normalized = resized_frame / 255.0
    frame_array = np.array([normalized])
    predictions = model.predict(frame_array)
    class_id = np.argmax(predictions)
    predicted_class = class_n[class_id]

    predictions1 = model1.predict(frame_array)
    class_id = np.argmax(predictions1)
    predicted_class1 = class_names[class_id]



    predicted_class_indices = np.argsort(predictions1[0])[::-1][:3]  # Indices of top 3 probabilities
    predicted_class_names = [class_names[index] for index in predicted_class_indices]
    confidences = [predictions1[0, index] for index in predicted_class_indices]

    # Calculate slip ratio dynamically
    # slip_ratios = [calculate_slip_ratio(vehicle_speed, tire_angular_velocity)
    #                        for _ in range(len(predicted_class_names))]

    #Calculate mu and w values
    mu_values = []
    for class_name in predicted_class_names:
        mu_range = friction_coefficients[class_name].split('-')
        mu = (float(mu_range[0]) + float(mu_range[1])) / 2 if len(mu_range) == 2 else float(mu_range[0])
        mu_values.append(mu)

    sum_confidences = sum(confidences)
    friction = sum((conf / sum_confidences) * mu for conf, mu in zip(confidences, mu_values))
    # friction_EKF = [kalman_filter(mu) for mu in mu_values]
    # surface_friction_EKF = f'{friction_EKF[0]:.2f}'
    surface_friction = f'{friction:.2f}'

    # class_data.append(predicted_class)
    # class_names_data.append(predicted_class_names)
    # confidences_data.append(confidences)
    # mu_values_data.append(mu_values)
    # slip_ratios_data.append(slip_ratios)
    # friction_data_EKF.append(surface_friction_EKF)
    # friction_data.append(surface_friction)

    # data = {
    #     'Predicted Class': class_data,
    #     'Class Names': class_names_data,
    #     'Confidences (P1, P2, P3)': confidences_data,
    #     'Mu Values (mu1, mu2, mu3)': mu_values_data,
    #     'Slip Ratios': slip_ratios_data,
    #     'Friction_VehicleDynamics': friction_data_EKF,
    #     'Surface Friction': friction_data
    # }
    # df = pd.DataFrame(data)
    # df.to_excel('Dashcam_Day_Dry_with_EKF.xlsx', index=False)

    # Add predictions and Slip Mu values to the dashboard
    # Mu_VehicleDynamics
  #   cv2.putText(
  #      frame,
  #      f"Mu_VehicleDynamics: {surface_friction_EKF}",
  #      (50, 50),
  #      font, font_scale, (0,255,0), thickness, cv2.LINE_AA
  #  )
    if(predicted_class=class_n[1]):


      cv2.putText(
          frame,
          f"{predicted_class} detected",
          (60, 50),
          font, font_scale, (0,255,0), thickness, cv2.LINE_AA
      )
    else:
        cv2.putText(
       frame,
       f"Mu_VehicleDynamics: {surface_friction_EKF}",
       (50, 50),
       font, font_scale, (0,255,0), thickness, cv2.LINE_AA
   )

    #WFL
    cv2.putText(
        frame,
        f"WFL: {predicted_class1}",
        (10, 20),  # Top-left corner of the dashboard
        font, font_scale, font_color, thickness, cv2.LINE_AA
    )
    cv2.putText(
        frame,
        f"Mu: {surface_friction}",
        (33, 23 + int(font_scale * 25)),  # Adjusted y-coordinate for the second line
        font, font_scale, font_color, thickness, cv2.LINE_AA
    )
    # WFR
    cv2.putText(
        frame,
        f"WFR: {predicted_class1}",
        (dashboard_width - 100, 20),  # Top-right corner of the dashboard
        font, font_scale, font_color, thickness, cv2.LINE_AA
    )
    cv2.putText(
        frame,
        f"Mu: {surface_friction}",
        (dashboard_width - 77, 23 + int(font_scale * 25)),  # Adjusted y-coordinate for the second line
        font, font_scale, font_color, thickness, cv2.LINE_AA
    )
    # WRL
    cv2.putText(
        frame,
        f"WRL: {predicted_class1}",
        (12, dashboard_height - 20),  # Bottom-left corner of the dashboard
        font, font_scale, font_color, thickness, cv2.LINE_AA
    )
    cv2.putText(
        frame,
        f"Mu: {surface_friction}",
        (35, dashboard_height - 17 + int(font_scale * 25)),  # Adjusted y-coordinate for the second line
        font, font_scale, font_color, thickness, cv2.LINE_AA
    )
    # WRR
    cv2.putText(
        frame,
        f"WRR: {predicted_class1}",
        (dashboard_width - 100, dashboard_height - 20),  # Bottom-right corner of the dashboard
        font, font_scale, font_color, thickness, cv2.LINE_AA
    )
    cv2.putText(
        frame,
        f"Mu: {surface_friction}",
        (dashboard_width - 77, dashboard_height - 17 + int(font_scale * 25)),  # Adjusted y-coordinate for the second line
        font, font_scale, font_color, thickness, cv2.LINE_AA
    )

    # Write the frame to the output video
    out.write(frame)


cap.release()
out.release()

print(f'Output video saved to: {output_video_path}')